# 4x4 Queens

### Validate a solution

In [6]:
def is_valid_solution(sample_sol):
    n = len(sample_sol)
    for i in range(n):
        for j in range(i + 1, n):
            # Check for row conflict
            if sample_sol[i] == sample_sol[j]:
                return False
            # Check for diagonal conflict
            if abs(sample_sol[i] - sample_sol[j]) == abs(i - j):
                return False
    return True

# Test the solution
sample_sol = [1, 2, 3, 0]
print(is_valid_solution(sample_sol))  # Outputs: False

sample_sol_valid = [1, 3, 0, 2]
print(is_valid_solution(sample_sol_valid))  # Outputs: True


False
True


### Visualising the board

In [7]:
def print_grid_board(solution):
    n = len(solution)
    for row in range(n):
        line = "|"
        for col in range(n):
            if solution[col] == n-row-1:
                line += " Q |"  # Place the queen
            else:
                line += "   |"  # Empty square
        print(line)
        print("-" * (n * 4 + 1))  # Draw the horizontal border

# Example Usage
sample_sol = [1, 3, 0, 2]
print_grid_board(sample_sol)



|   | Q |   |   |
-----------------
|   |   |   | Q |
-----------------
| Q |   |   |   |
-----------------
|   |   | Q |   |
-----------------


### Generate a random solution

In [8]:
import random

def generate_random_solution(n):
    """
    Generate a random solution for the N-Queens problem with possible duplicates.
    """
    solution = [random.randint(0, n - 1) for _ in range(n)]  # Randomly assign rows for each column
    return solution

In [9]:
rand_sol = generate_random_solution(8)
print(rand_sol)
print_grid_board(rand_sol)

[1, 5, 2, 7, 6, 2, 1, 2]
|   |   |   | Q |   |   |   |   |
---------------------------------
|   |   |   |   | Q |   |   |   |
---------------------------------
|   | Q |   |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   |   |   |
---------------------------------
|   |   | Q |   |   | Q |   | Q |
---------------------------------
| Q |   |   |   |   |   | Q |   |
---------------------------------
|   |   |   |   |   |   |   |   |
---------------------------------


In [10]:
print(is_valid_solution(rand_sol))

False


### Complete enumeration using DFS

#### DFS without prune

In [11]:
import copy

def DFS(master, sol, idx, branches, size):
    for i in range(size):
        temp_sol = sol
        temp_sol[idx] = i
        
        if idx <size -1:
            DFS(master, temp_sol, idx+1, branches, size)
        else:
            branches[0]+=1
            if is_valid_solution(temp_sol):
                valid_sol = copy.deepcopy(temp_sol)
                master.append(valid_sol)
#                 print(valid_sol)
                

# DFS([0,0,0,0],0)

#### DFS with prune

In [12]:
import copy

def DFS_prune(master, sol, idx, branches,size):
    for i in range(size):
        temp_sol = sol
        temp_sol[idx] = i
        
        if idx <size-1 and idx >=1 and temp_sol[idx] in temp_sol[0:idx]:
            continue
        elif idx <size-1:
            DFS_prune(master, temp_sol, idx+1, branches, size)
        else:
            branches[0]+=1
            if is_valid_solution(temp_sol):
                valid_sol = copy.deepcopy(temp_sol)
                master.append(valid_sol)
#                 print(valid_sol)
                
# DFS([0,0,0,0],0)

In [13]:
size = 8
branches = [0]
master = []
start_sol = [0]*size
start_time = time.time()
DFS(master, start_sol, 0, branches, size)
end_time = time.time()
print("Master solutions size:", len(master))
print("Checked Branches:", branches[0])
print("DFS Time:", end_time - start_time, "seconds")

Master solutions size: 92
Checked Branches: 16777216
DFS Time: 19.26918601989746 seconds


In [14]:
size = 8
branches = [0]
master = []
start_sol = [0]*size
start_time = time.time()
DFS_prune(master, start_sol, 0, branches, size)
end_time = time.time()
print("Master solutions size:", len(master))
print("Checked Branches:", branches[0])
print("DFS Time:", end_time - start_time, "seconds")

Master solutions size: 92
Checked Branches: 322560
DFS Time: 0.5702459812164307 seconds


In [186]:
for sol in master:
    print_grid_board(sol)
    print("---------Solution Completed---------")

|   |   | Q |   |   |   |   |   |
---------------------------------
|   |   |   |   |   | Q |   |   |
---------------------------------
|   |   |   | Q |   |   |   |   |
---------------------------------
|   | Q |   |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   |   | Q |
---------------------------------
|   |   |   |   | Q |   |   |   |
---------------------------------
|   |   |   |   |   |   | Q |   |
---------------------------------
| Q |   |   |   |   |   |   |   |
---------------------------------
---------Solution Completed---------
|   |   | Q |   |   |   |   |   |
---------------------------------
|   |   |   |   | Q |   |   |   |
---------------------------------
|   | Q |   |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   |   | Q |
---------------------------------
|   |   |   |   |   | Q |   |   |
---------------------------------
|   |   |   | Q |   |   |   |   |
---------------------------------
|   |   |  

### Constraint Programming - Solver OR tools

In [17]:
from ortools.sat.python import cp_model

def cp_solve(n):
    # Create the model
    model = cp_model.CpModel()

    # Define variables: one integer variable per queen's row position
    queens = [model.NewIntVar(0, n - 1, f'queen_{col}') for col in range(n)]

    # Add constraints
    model.AddAllDifferent(queens)  # No two queens in the same row
    model.AddAllDifferent([queens[i] + i for i in range(n)])  # Left-to-right diagonals
    model.AddAllDifferent([queens[i] - i for i in range(n)])  # Right-to-left diagonals

    # Create the solver
    solver = cp_model.CpSolver()

    # Capture solutions
    solutions = []

    class SolutionPrinter(cp_model.CpSolverSolutionCallback):
        def __init__(self, queens):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self.queens = queens

        def OnSolutionCallback(self):
            solution = [self.Value(self.queens[col]) for col in range(n)]
            solutions.append(solution)

    # Solve the model
    solution_printer = SolutionPrinter(queens)
    solver.SearchForAllSolutions(model, solution_printer)

    return solutions


In [19]:
size = 8
start_time = time.time()
master=cp_solve(size)
end_time = time.time()
print("Master solutions size:", len(master))
print("Constraint Programming Time:", end_time - start_time, "seconds")

Master solutions size: 92
Constraint Programming Time: 0.01803874969482422 seconds


In [20]:
for sol in master:
    print_grid_board(sol)
    print("---------Solution Completed---------")

|   | Q |   |   |   |   |   |   |
---------------------------------
|   |   |   |   |   |   |   | Q |
---------------------------------
|   |   |   |   |   | Q |   |   |
---------------------------------
| Q |   |   |   |   |   |   |   |
---------------------------------
|   |   | Q |   |   |   |   |   |
---------------------------------
|   |   |   |   | Q |   |   |   |
---------------------------------
|   |   |   |   |   |   | Q |   |
---------------------------------
|   |   |   | Q |   |   |   |   |
---------------------------------
---------Solution Completed---------
|   |   |   |   |   | Q |   |   |
---------------------------------
|   |   |   | Q |   |   |   |   |
---------------------------------
|   |   |   |   |   |   | Q |   |
---------------------------------
| Q |   |   |   |   |   |   |   |
---------------------------------
|   |   | Q |   |   |   |   |   |
---------------------------------
|   |   |   |   | Q |   |   |   |
---------------------------------
|   | Q |  